In [ ]:
# To Train models, upload the pedalnet repository to your Google Drive account and run
# the following code. In Colab options, go to "Runtime", "Change Runtime Type",
# and change hardware accelerator to "TPU".
#
# Note: User needs to modify the train.py script to use TPUs. Uncomment line as instructed in the file notes.
#
# Note: Having difficulty saving and loading TPU model after converting to CPU.
#       It can be done, but currently needs some work.

!pip3 install torchvision
!pip install pytorch-lightning

!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

In [ ]:
import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
os.chdir('/content/drive/ENTER_YOUR_GOOGLE_DRIVE_PATH_HERE/pedalnet')
os.getcwd()


In [ ]:
!python3 "prepare_data.py" "data/ts9_test1_in_FP32.wav" "data/ts9_test1_out_FP32.wav"


In [ ]:

import os
assert os.environ['COLAB_TPU_ADDR']
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=1500 --learning_rate=3e-3

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
